In [95]:
#importing modules
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [9]:
#importing dataset
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [3]:
# looking training data
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [4]:
# looking test data
test_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


In [5]:
train_df.columns

Index(['Unnamed: 0', 'index', 'loan_amnt', 'int_rate', 'installment',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_ol

In [13]:
#shape of training data
train_df.shape

(12180, 84)

In [12]:
#shape of test data
test_df.shape

(4702, 84)

In [52]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 84 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   12180 non-null  float64
 1   int_rate                    12180 non-null  float64
 2   installment                 12180 non-null  float64
 3   home_ownership              12180 non-null  object 
 4   annual_inc                  12180 non-null  float64
 5   verification_status         12180 non-null  object 
 6   loan_status                 12180 non-null  object 
 7   pymnt_plan                  12180 non-null  object 
 8   dti                         12180 non-null  float64
 9   delinq_2yrs                 12180 non-null  float64
 10  inq_last_6mths              12180 non-null  float64
 11  open_acc                    12180 non-null  float64
 12  pub_rec                     12180 non-null  float64
 13  revol_bal                   121

In [10]:
#removing unnessary columns from training and testing set
train_df.drop(columns=['Unnamed: 0', 'index'], inplace=True)
test_df.drop(columns=['Unnamed: 0', 'index'], inplace=True)

In [11]:
train_df['loan_status'].value_counts()

low_risk     6090
high_risk    6090
Name: loan_status, dtype: int64

## Train and test set

In [15]:
# create feature matrix (X) #selecting all columns except 'loan_status'
X_train=train_df.drop('loan_status', axis=1) 
X_test=test_df.drop('loan_status', axis=1) 

# create response vector (y) ##selecting 'loan_status'
y_train=train_df['loan_status'].values  
y_test=test_df['loan_status'].values  

In [18]:
y_train

array(['low_risk', 'low_risk', 'low_risk', ..., 'high_risk', 'high_risk',
       'high_risk'], dtype=object)

## Converting categorical data to numeric

In [19]:
# Convert categorical data to numeric
X_train_dump=pd.get_dummies(X_train)
X_test_dump=pd.get_dummies(X_test)

In [20]:
#shape of test and train after pd.get_dummies
print(X_train_dump.shape)
print(X_test_dump.shape)

(12180, 92)
(4702, 91)


In [55]:
#adding the missing columns in test data(no dept_settlement_flag)
X_test_dump['debt_settlement_flag_Y']=0

In [70]:
 # Converting output labels to 0 and 1
y_label = LabelEncoder().fit_transform(y_train)
y_label

array([1, 1, 1, ..., 0, 0, 0])

### I think that random forest will be better than the logistic regression before standard scaler

## LogisticRegression

In [26]:
# Create a Logistic Regression model on the unscaled data
lr= LogisticRegression()

# Fit the model to the data
lr.fit(X_train_dump, y_train)

In [72]:
#printing the accuracy score
print(f'Training Score: {lr.score(X_train_dump, y_train)}')
print(f'Testing Score: {lr.score(X_test_dump, y_test)}')

Training Score: 0.6508210180623973
Testing Score: 0.5168013611229264


In [76]:
#score of training set using cross_val_score
cross_val_score(lr, X_train_dump, y_train, cv=5, scoring='accuracy')

array([0.58415435, 0.64039409, 0.67323481, 0.658867  , 0.67200328])

In [79]:
#predection on the train set and confusion matrix
#cross_val_predict performs k-fold cross-validation but instead of returning evaluation scores it return the prediction made in each fold.
y_train_pred=cross_val_predict(lr, X_train_dump, y_train, cv=5)
confusion_matrix(y_train, y_train_pred)

array([[3511, 2579],
       [1736, 4354]])

In [102]:
#predection on the test set and confusion matrix
y_pred=lr.predict(X_test_dump)
confusion_matrix(y_test, y_pred)

array([[ 712, 1639],
       [ 633, 1718]])

In [81]:
#classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.50      0.21      0.30      2351
    low_risk       0.50      0.79      0.61      2351

    accuracy                           0.50      4702
   macro avg       0.50      0.50      0.45      4702
weighted avg       0.50      0.50      0.45      4702



## Random Forest Classifier

In [98]:
#Random Forest classifier model
rf=RandomForestClassifier(random_state=1, n_estimators=500)
rf.fit(X_train_dump, y_train)

RandomForestClassifier(n_estimators=500, random_state=1)

In [99]:
#printing the accuracy score
print(f'Training Score: {rf.score(X_train_dump, y_train)}')
print(f'Testing Score: {rf.score(X_test_dump, y_test)}')

Training Score: 1.0
Testing Score: 0.6433432581880051


In [100]:
#score of training set using cross_val_score
cross_val_score(rf, X_train_dump, y_train, cv=5, scoring='accuracy')

array([0.65763547, 0.74712644, 0.79392447, 0.80829228, 0.82060755])

In [101]:
#predection on the train set and confusion matrix
#cross_val_predict performs k-fold cross-validation but instead of returning evaluation scores it return the prediction made in each fold.
y_train_pred=cross_val_predict(rf, X_train_dump, y_train, cv=5)
confusion_matrix(y_train, y_train_pred)

array([[4601, 1489],
       [1367, 4723]])

In [103]:
#predection on the test set and confusion matrix
y_pred_rf=rf.predict(X_test_dump)
confusion_matrix(y_test, y_pred_rf)

array([[1888,  463],
       [1214, 1137]])

In [104]:
#classification report
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

   high_risk       0.61      0.80      0.69      2351
    low_risk       0.71      0.48      0.58      2351

    accuracy                           0.64      4702
   macro avg       0.66      0.64      0.63      4702
weighted avg       0.66      0.64      0.63      4702



## I am wright that the random forest is better than the logistic regression before scaling because scaling will not affect random forest

## Scaling the data using StandardScalar

In [84]:
# Create a StandardScaler() model and fit it to the training data
X_scaler = StandardScaler().fit(X_train_dump)

In [85]:
# Transform the training and testing data by using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train_dump)
X_test_scaled = X_scaler.transform(X_test_dump)

## LogisticRegression after scaling data

In [86]:
# Creating a Logistic Regression model on the scaled data
lr_s= LogisticRegression()

# Fit the model to the data
lr_s.fit(X_train_scaled, y_train)

LogisticRegression()

In [87]:
#printing the accuracy score
print(f'Training Score: {lr_s.score(X_train_scaled, y_train)}')
print(f'Testing Score: {lr_s.score(X_test_scaled, y_test)}')

Training Score: 0.7078817733990148
Testing Score: 0.767333049766057


In [88]:
#score of training set using cross_val_score
cross_val_score(lr_s, X_train_scaled, y_train, cv=5, scoring='accuracy')

array([0.66871921, 0.6728243 , 0.7089491 , 0.71305419, 0.71428571])

In [89]:
#predection on the train set and confusion matrix
#cross_val_predict performs k-fold cross-validation but instead of returning evaluation scores it return the prediction made in each fold.
y_train_pred=cross_val_predict(lr_s, X_train_scaled, y_train, cv=5)
confusion_matrix(y_train, y_train_pred)

array([[3906, 2184],
       [1524, 4566]])

In [92]:
#predection on the test set and confusion matrix
y_pred_scaled=lr_s.predict(X_test_scaled)
confusion_matrix(y_test, y_pred_scaled)

array([[1789,  562],
       [ 532, 1819]])

In [93]:
#classification report
print(classification_report(y_test, y_pred_scaled))

              precision    recall  f1-score   support

   high_risk       0.77      0.76      0.77      2351
    low_risk       0.76      0.77      0.77      2351

    accuracy                           0.77      4702
   macro avg       0.77      0.77      0.77      4702
weighted avg       0.77      0.77      0.77      4702



## Random Forest Classifier after scaling

In [107]:
#Random Forest classifier model
rf_s=RandomForestClassifier(random_state=1, n_estimators=500)
rf_s.fit(X_train_scaled, y_train)

RandomForestClassifier(n_estimators=500, random_state=1)

In [108]:
#printing the accuracy score
print(f'Training Score: {rf_s.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rf_s.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.6420672054444917


In [109]:
#score of training set using cross_val_score
cross_val_score(rf_s, X_train_scaled, y_train, cv=5, scoring='accuracy')

array([0.65763547, 0.74835796, 0.79351396, 0.80870279, 0.82142857])

In [110]:
#predection on the train set and confusion matrix
#cross_val_predict performs k-fold cross-validation but instead of returning evaluation scores it return the prediction made in each fold.
y_train_pred=cross_val_predict(rf_s, X_train_scaled, y_train, cv=5)
confusion_matrix(y_train, y_train_pred)

array([[4601, 1489],
       [1362, 4728]])

In [111]:
#predection on the test set and confusion matrix
y_pred_rfs=rf.predict(X_test_scaled)
confusion_matrix(y_test, y_pred_rfs)

array([[1890,  461],
       [1222, 1129]])

In [112]:
#classification report
print(classification_report(y_test, y_pred_rfs))

              precision    recall  f1-score   support

   high_risk       0.61      0.80      0.69      2351
    low_risk       0.71      0.48      0.57      2351

    accuracy                           0.64      4702
   macro avg       0.66      0.64      0.63      4702
weighted avg       0.66      0.64      0.63      4702



### After scaling logistic regression is far better than random forest because scaling has a huge impact on Logistic Regression